In [1]:
import torch
from tqdm.notebook import trange
from tradezoo.agent import Actor, Agent, Critic
from tradezoo.game import Game, Client, SineWave, Trader
from tradezoo.market import Account, Market
from tradezoo.plots import balance_plot, decision_plot, reward_plot, trades_plot, uncertainty_plot, utility_plot
from tradezoo.trainer import Trainer

In [2]:
def make_agent():
    actor = Actor()
    critic = Critic()
    return Agent(
        actor=actor,
        actor_optimizer=torch.optim.Adam(actor.parameters(), lr=1e-4),
        critic=critic,
        critic_optimizer=torch.optim.Adam(critic.parameters(), lr=2e-2),
        discount_factor=0.99,
        uncertainty=1e-3,
    )


num_traders = 1
trader_accounts = [
    Account(cash_balance=2048, asset_balance=2048) for _ in range(num_traders)
]
client_accounts = [
    Account(cash_balance=float("inf"), asset_balance=float("inf"))
    for _ in range(num_traders)
]
price_process = 1 + SineWave(period=256) * 0.2
traders = [
    Trader(
        agent=make_agent(),
        account=trader_account,
        client=Client(
            account=client_account,
            for_account=trader_account,
            ask_process=price_process * 1.1,
            bid_process=price_process * 0.9,
        ),
    )
    for trader_account, client_account in zip(trader_accounts, client_accounts)
]
game = Game.new(
    market=Market.from_accounts(trader_accounts + client_accounts),
    traders=traders,
)
trainer = Trainer.new(game=game, replay_buffer_capacity=16, batch_size=16)
turn_results = [trainer.turn_() for _ in trange(4096)]

  0%|          | 0/4096 [00:00<?, ?it/s]

In [3]:
trades_plot(turn_results)

In [4]:
reward_plot(turn_results)

In [5]:
uncertainty_plot(turn_results)

In [6]:
balance_plot(turn_results)

In [7]:
utility_plot(traders[0].agent)

In [8]:
decision_plot(traders[0].agent)